In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

import pandas as pd

from hist import Hist

import babar_analysis_tools as bat
import myPIDselector

import os 

In [ ]:
start = time.time()

# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'

# At home (Bellis)
topdir = '/home/bellis/babar_data/bnv_plambda'


### Read in SP- includes both signal and background 
#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'
#filename = f'{topdir}/Background_and_signal_SP_modes_All_runs.parquet'

### convert files from a parquet file to an awkward array
start = time.time()
ak_sp = ak.from_parquet(filename)
print(f"Took {time.time() - start} s")


# Read in data
#filename = f'{topdir}/Data_Only_Run_1_BLINDED.parquet'
#start = time.time()
#ak_data = ak.from_parquet(filename)
#print(f"Took {time.time() - start} s")


In [ ]:
spmask = ak_sp['spmode']=='-999'

ak_sp = ak_sp[spmask]

In [ ]:
q = (ak_sp['Bd2Lund'][:,0])/np.abs(ak_sp['Bd2Lund'][:,0])
q

In [ ]:
eps = myPIDselector.PIDselector("e")
pps = myPIDselector.PIDselector("p")
pips = myPIDselector.PIDselector("pi")
Kps = myPIDselector.PIDselector("K")
mups = myPIDselector.PIDselector("mu")

In [ ]:
lamd1idx = ak_sp['Lambda0d1Idx']
lamd1Lund = ak_sp['Lambda0d1Lund']

print(f'lamd1idx\n{lamd1idx}')
print(f'lamd1Lund\n{lamd1Lund}')


d2idx = ak_sp['Bd2Idx']
d2Lund = ak_sp['Bd2Lund']

print(f'd2idx\n{d2idx}')
print(f'd2Lund\n{d2Lund}')

trkidx_proton = ak_sp['pTrkIdx']
trk_selector_map = ak_sp['pSelectorsMap']
print()

print("trk_selector_map")
print(trk_selector_map)

# Do this for all the tracks
pbits = bat.calculate_bits_for_PID_selector(trkidx_proton, trk_selector_map, verbose=1)
print(f'pbits:\n{pbits}')
print(f'pbits[d2idx]:\n{pbits[d2idx]}')
print()

selector = 'LooseKMProtonSelection'
print(f"Now trying to create a mask with {selector}")

mask_bool = bat.mask_PID_selection(pbits[d2idx], selector, pps)
print(f'mask_book: {mask_bool}')
print()
passing_pbits = pbits[d2idx][mask_bool]
print("\nMasked bits of the daughters...")
print(f'{passing_pbits}')
print()

print("\nMasked bits of the not daughters...")
passing_pbits_not_daughter = pbits[~d2idx][mask_bool]
print(f'{passing_pbits_not_daughter}')
print()

# Jim's solution

https://stackoverflow.com/questions/78657390/accessing-elements-of-an-awkward-array-that-are-not-a-passed-in-index/78659051#78659051

In [ ]:
def indices_to_booleans(indices, array_to_slice):
    whole_set, in_set = ak.unzip(ak.cartesian([
        ak.local_index(array_to_slice), indices
    ], nested=True))
    return ak.any(whole_set == in_set, axis=-1)

In [ ]:
jet_lepton_indices = ak.Array([[0, 2], [1], [2,3]])
print(f'jet_lepton_indices\n{jet_lepton_indices}\n')

lepton_flags = ak.Array([[0, 10, 20, 30], [0, 10, 20, 30], [0, 10, 20, 30, 40]])
print(f'lepton_flags\n{lepton_flags}\n')

In [ ]:
idx = indices_to_booleans(jet_lepton_indices, lepton_flags)

idx

In [ ]:
lepton_flags[idx]

In [ ]:
lepton_flags[~idx]

With our code.

In [ ]:
lamd1idx = ak_sp['Lambda0d1Idx']
lamd1Lund = ak_sp['Lambda0d1Lund']

print(f'lamd1idx\n{lamd1idx}')
print(f'lamd1Lund\n{lamd1Lund}')


d2idx = ak_sp['Bd2Idx']
d2Lund = ak_sp['Bd2Lund']

print(f'd2idx\n{d2idx}')
print(f'd2Lund\n{d2Lund}')
print()

trkidx_proton = ak_sp['pTrkIdx']
print(f"trkidx_proton\n{trkidx_proton}")
print()

lamd1_trkidx = trkidx_proton[lamd1idx]
print(f"lamd1_trkidx\n{lamd1_trkidx}\n")

trk_selector_map = ak_sp['pSelectorsMap']
print(f"trk_selector_map\n{trk_selector_map}\n")

lamproton_selector_map = trk_selector_map[lamd1_trkidx]
print(lamproton_selector_map)
print()

idx = indices_to_booleans(lamd1_trkidx, trk_selector_map)
print(idx)
print(trk_selector_map[idx])
print(trk_selector_map[~idx])
